<a href="https://colab.research.google.com/github/MStamirski/Wind-prediction/blob/main/WIND_only_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook contains a program, which predicts a speed and direction of wind in location selected by user.

It is based on data provided on the website https://map.neweuropeanwindatlas.eu/
Program downloads data from area around chosen coordinates. These data consist of wind speed (in m/s) and direction (in degrees) in nodes of gride, distant from each other by 3 kilometers.

Four periods of prediction are assumed: 1.5 hour, 3 hours, 6 hours and 12 hours.
For each of them a separate algorithm was developped. 

In 1.5 and 3 hours prediction, calculations use matrix of 9x9 nodes (with chosen location placed in the middle), and MLPRegressor model of neural network.

In 6 hours prediction, calculations use the matrix of 15x15 nodes and Sequential model of neural network.

In 12 hours prediction, calculations use also matrix 15x15 and neural network model with convolutional layer.

All hyperparameters were optimized for each model and prediction period separately.

User can chose years from which traning and testing data are collected (both from range 1989-2018).

As a result program shows MAE and MAPE metrics, calculated for predictions as well as for baseline model, which gives data from a day of prediction as a prediction. The negative difference between metrics of these two models signifies, that predictions are valuable.

Comparisons are also showed on diagrams and animations.

# Functions for loading data

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import math
import urllib

In [ ]:
def get_wind_data(center_lat, 
                  center_lon, 
                  area_radius_km,
                  year,
                  header):
  
  print(header)

  variable1 = 'WD10'  # wind direction
  variable2 = 'WS10'  # wind speed

  # The approximate conversions are: Latitude: 1 deg = 110.574 km. Longitude: 1 deg = 111.320*cos(latitude) km.
  # area height in degrees:
  ver_radius_deg = area_radius_km / 110.574
  # area width in degrees:
  hor_radius_deg = area_radius_km / (111.320 * math.cos(np.radians(center_lat)))

  southBoundLatitude = center_lat - ver_radius_deg
  northBoundLatitude = center_lat + ver_radius_deg
  westBoundLongitude = center_lon - hor_radius_deg
  eastBoundLongitude = center_lon + hor_radius_deg

  dt_start = str(year) + '-01-01T00:00:00'
  dt_stop = str(year) + '-12-31T23:30:00'

  url = 'https://wps.neweuropeanwindatlas.eu/api/mesoscale-ts/v1/get-data-bbox'
  url += '?southBoundLatitude=' + str(southBoundLatitude)
  url += '&northBoundLatitude=' + str(northBoundLatitude)
  url += '&westBoundLongitude=' + str(westBoundLongitude)
  url += '&eastBoundLongitude=' + str(eastBoundLongitude)
  url += '&variable=' + str(variable1)
  url += '&variable=' + str(variable2)
  url += '&dt_start=' + dt_start
  url += '&dt_stop=' + dt_stop
  
  data = urllib.request.urlopen(url).read()

  df = xr.open_dataset(data).to_dataframe()

  print("\nDone ! --------------------------------\n")

  return df

# Functions for data preprocessing

In [ ]:
import time

In [ ]:
def columns_preparation(data):
  data.reset_index(inplace=True)
  data.drop(columns=['south_north', 'west_east', 'crs'], inplace=True)
  data.fillna(0, inplace=True)

In [ ]:
def nodes_preparation(data):
  first_data = data.loc[0, 'time']
  nodes=data[data['time']==first_data][['XLAT', 'XLON']].sort_values(by=['XLAT','XLON']).reset_index(drop=True)
  nodes['node_nr'] = nodes.index
  return nodes

In [ ]:
def nodes_to_matrix(nodes):
  matrix = []
  nodes_set = nodes.copy()            # nodes not chosen to matrix yet
  
  while len(nodes_set)>0:
    row = []
    nodes_subset = nodes_set.copy()   # nodes searched for current row

    while len(nodes_subset)>0:
      node = nodes_subset[nodes_subset['XLAT']==nodes_subset['XLAT'].min()]
      row.append(node['node_nr'].values[0])

      nodes_subset = nodes_subset[nodes_subset['XLON']>node['XLON'].values[0]]
      nodes_set.drop(index=node.index, inplace=True)
    
    matrix.append(row)
  
  return matrix

In [ ]:
def coord_to_nodes(nodes):
  nodes_dict = {}
  for ind in nodes.index:
    nodes_dict[(nodes.loc[ind,'XLAT'], nodes.loc[ind,'XLON'])] = nodes.loc[ind,'node_nr']
  return nodes_dict

In [ ]:
def get_node_pos(matrix, coord_to_nodes, xlat, xlon):
  node_nr = coord_to_nodes[(xlat, xlon)]
  array = np.array(matrix)
  row, col = np.where(array==node_nr)
  return node_nr, row[0], col[0]

In [ ]:
def data_with_nodes_matrix(dataset, matrix, coord):
  start = mlsec = time.time()
  data = dataset.copy().sort_values(by=['time'])

  xlat_list = data['XLAT'].tolist()
  xlon_list = data['XLON'].tolist()

  node_nr_list = []
  node_row_list = []
  node_col_list = []

  for ind in range(len(data)):
    xlat = xlat_list[ind]
    xlon = xlon_list[ind]
    nr, row, col = get_node_pos(matrix, coord, xlat, xlon)
    node_nr_list.append(nr)
    node_row_list.append(row)
    node_col_list.append(col)

    if ind % 10000 == 0:
      print(f"\rindex: {ind+1} of {len(data)}, time of step: {time.time()-mlsec}, total time: {time.time()-start}", end="")
      mlsec = time.time()
  
  data['node_nr'] = node_nr_list
  data['node_row'] = node_row_list
  data['node_col'] = node_col_list
  data.drop(columns=['XLAT', 'XLON'], inplace=True)

  print(f"\nAdding columns from lists, time of step: {time.time()-mlsec}, total time: {time.time()-start}")
  return data

In [ ]:
def prepare_dataframe(df, header):
  print(header)
  print("Source dataset shape: ", df.shape)
  columns_preparation(df)
  nodes = nodes_preparation(df)
  matrix = nodes_to_matrix(nodes)
  print(f"Grid of nodes - rows: {len(matrix)}, columns: {len(matrix[0])}")
  coord = coord_to_nodes(nodes)
  print("\nInserting nodes coordinates")
  df = data_with_nodes_matrix(df, matrix, coord)
  print("\nDone ! --------------------------------\n")
  return df

# Functions for preparation of training and testing datasets

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_training_nodes(df_prep, x_coord, center_coord):

  X_col = [coord[1] for coord in x_coord]
  X_row = [coord[0] for coord in x_coord]

  maxrow = df_prep['node_row'].max()
  maxcol = df_prep['node_col'].max()
  all_row = []
  all_col = []
  for r in range(maxrow+1):
    for c in range(maxcol+1):
      all_col.append(c)
      all_row.append(r)

  plt.figure(figsize=(16, 6))
  plt.title("Selected nodes")
  plt.xlabel("Columns")
  plt.ylabel("Rows")
  plt.grid(visible=True, which='major', axis='both')

  all_nodes = plt.scatter(all_col, all_row, color='gray', s=80)
  y_node = plt.scatter(center_coord[1], center_coord[0], color='blue', s=400)
  x_nodes = plt.scatter(X_col, X_row, color='red', s=150)

  node = df_prep[(df_prep['node_row']==center_coord[0]) & (df_prep['node_col']==center_coord[1])]['node_nr'].values[0]
  plt.annotate(node, xy=(center_coord[1]+0.2, center_coord[0]-0.2))
  for x in range(len(x_coord)):
    node = df_prep[(df_prep['node_row']==x_coord[x][0]) & (df_prep['node_col']==x_coord[x][1])]['node_nr'].values[0]
    plt.annotate(node, xy=(x_coord[x][1]+0.1, x_coord[x][0]+0.1))

  plt.legend((all_nodes, x_nodes, y_node), ('unused nodes', 'X nodes', 'y node'), loc='upper left', shadow=True)
  plt.show()

In [ ]:
def create_dataset(df_prep, center_coord, nodes_radius=4, internal_radius=0, pred_steps=1, show=False):
  start = mlsec = time.time()

  # area of nodes
  bot_row = max(0, center_coord[0]-nodes_radius)
  top_row = min(df_prep['node_row'].max(), center_coord[0]+nodes_radius)
  left_col = max(0, center_coord[1]-nodes_radius)
  right_col = min(df_prep['node_col'].max(), center_coord[1]+nodes_radius)

  # list of tuples with nodes coordinates
  x_coord = []
  x_rows = [row for row in range(bot_row, top_row+1)]
  x_cols = [col for col in range(left_col, right_col+1)]
  for r in range(len(x_rows)):
    for c in range(len(x_cols)):
      if ((x_cols[c] < center_coord[1]+1-internal_radius) or (x_cols[c] > center_coord[1]-1+internal_radius)) or ((x_rows[r] < center_coord[0]+1-internal_radius) or (x_rows[r] > center_coord[0]-1+internal_radius)):
        x_coord.append((x_rows[r], x_cols[c]))
 
  if show: plot_training_nodes(df_prep, x_coord, center_coord)

  x_nr = len(x_coord)

  # lists of wind speed and direction for y
  y_data = df_prep[(df_prep['node_row']==center_coord[0]) & (df_prep['node_col']==center_coord[1])].sort_values(by=['time'])
  y_WS = y_data['WS10'].tolist()
  y_WD = y_data['WD10'].tolist()

  # lists of wind speed and direction for each x
  xi_WS = []
  xi_WD = []
  for i in range(x_nr):
    x_data = df_prep[(df_prep['node_row']==x_coord[i][0]) & (df_prep['node_col']==x_coord[i][1])].sort_values(by=['time'])
    x_WS = x_data['WS10'].tolist()
    x_WD = x_data['WD10'].tolist()
    xi_WS.append(x_WS)
    xi_WD.append(x_WD)

  # creating dictionary of testing data
  test_rows = []
  rows_nr = len(y_WS)-pred_steps
  for ind in range(rows_nr):
    test_row = {}
    test_row['y_WS'] = y_WS[ind+pred_steps]
    test_row['y_WD_sin'] = math.sin(np.radians(y_WD[ind+pred_steps]))
    test_row['y_WD_cos'] = math.cos(np.radians(y_WD[ind+pred_steps]))

    for x in range(x_nr):
      test_row['X'+str(x+1)+'_WS'] = xi_WS[x][ind]
      test_row['X'+str(x+1)+'_WD_sin'] = math.sin(np.radians(xi_WD[x][ind]))
      test_row['X'+str(x+1)+'_WD_cos'] = math.cos(np.radians(xi_WD[x][ind]))

    if (ind % 1000 == 0):
      print(f"\rindex: {ind+1} of {rows_nr}, time of step: {time.time()-mlsec}, total time: {time.time()-start}", end="")
      mlsec = time.time()    
    
    test_rows.append(test_row)

  df_test = pd.DataFrame.from_dict(test_rows)
  print(f"\ncreating dataframe from dict, time of step: {time.time()-mlsec}, total time: {time.time()-start}")

  return df_test

In [ ]:
def create_baseline_dataset(df_prep, center_coord, pred_steps):

  # lists of wind speed and direction for y
  y_data = df_prep[(df_prep['node_row']==center_coord[0]) & (df_prep['node_col']==center_coord[1])].sort_values(by=['time'])
  y_WS = y_data['WS10'].tolist()
  y_WD = y_data['WD10'].tolist()

  # creating dictionary of testing data
  test_rows = []
  rows_nr = len(y_WS)-pred_steps
  for ind in range(rows_nr):
    test_row = {}
    test_row['y_WS'] = y_WS[ind]
    test_row['y_WD'] = y_WD[ind]
     
    test_rows.append(test_row)

  df_base = pd.DataFrame.from_dict(test_rows)

  return df_base

In [ ]:
def create_prediction_datasets(df_prep, y_node, header, baseline=False):
  
  print(header)

  # size and shape of nodes matrix optimalized before
  
  print("\nCreating prediction dataset: 1.5 hour ahead")
  df_1 = create_dataset(df_prep, y_node, nodes_radius=4, internal_radius=0, pred_steps=3, show=False)
  print("\nCreating prediction dataset: 3 hour ahead")
  df_3 = create_dataset(df_prep, y_node, nodes_radius=4, internal_radius=0, pred_steps=6, show=False)
  print("\nCreating prediction dataset: 6 hour ahead")
  df_6 = create_dataset(df_prep, y_node, nodes_radius=7, internal_radius=0, pred_steps=12, show=False)
  print("\nCreating prediction dataset: 12 hour ahead")
  df_12 = create_dataset(df_prep, y_node, nodes_radius=7, internal_radius=0, pred_steps=24, show=False)

  # baseline only for testing datasets (not training)
  if baseline:
    print("\nCreating baseline datasets")
    base_1 = create_baseline_dataset(df_prep, y_node, pred_steps=3)
    base_3 = create_baseline_dataset(df_prep, y_node, pred_steps=6)
    base_6 = create_baseline_dataset(df_prep, y_node, pred_steps=12)
    base_12 = create_baseline_dataset(df_prep, y_node, pred_steps=24)

  print("\nDone ! --------------------------------\n")

  if baseline: 
    return df_1, df_3, df_6, df_12, base_1, base_3, base_6, base_12
  else:
    return df_1, df_3, df_6, df_12

# Functions for predictions

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from keras import models
from keras import layers
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

In [ ]:
def get_data(df, scaler):
  
  X = df[df.columns[3:]]
  y = df[df.columns[:3]]

  X_scaled = scaler.fit_transform(X)
  y_scaled = scaler.fit_transform(y)

  return X_scaled, y_scaled

In [ ]:
def get_baseline(df, scaler):
  y_scaled = scaler.fit_transform(df)
  return y_scaled

In [ ]:
def MAE_deg(y_true, y_pred):
  count = 0
  errsum = 0
  for y1, y2 in zip(y_true, y_pred):
    count += 1
    ymax = np.where(y1>y2, y1, y2)
    ymin = np.where(y1>y2, y2, y1)
    err1 = ymax-ymin
    err2 = ymin + (360-ymax)
    errsum += min(err1, err2)
  return errsum/count

In [ ]:
def MAPE_deg(y_true, y_pred):
  perrsum = 0
  for y1, y2 in zip(y_true, y_pred):
    ymax = np.where(y1>y2, y1, y2)
    ymin = np.where(y1>y2, y2, y1)
    err1 = ymax-ymin
    err2 = ymin + (360-ymax)
    perr = (min(err1, err2) / 180) * 100
    perrsum += perr
  return perrsum / len(y_true)

In [ ]:
def make_prediction(model, scaler, X_test, y_test, scaler_base, y_base):

  y_pred = model.predict(X_test)

  y_pred_unscaled = scaler.inverse_transform(y_pred)
  y_test_unscaled = scaler.inverse_transform(y_test)
  y_base_unscaled = scaler_base.inverse_transform(y_base)

  WS_pred_unscaled = [y[0] for y in y_pred_unscaled]
  WS_test_unscaled = [y[0] for y in y_test_unscaled]
  WS_base_unscaled = [y[0] for y in y_base_unscaled]

  WS_pred_scaled = [3 * y[0] for y in y_pred]
  WS_test_scaled = [3 * y[0] for y in y_test]
  WS_base_scaled = [3 * y[0] for y in y_base]

  WD_sin_pred_unscaled = [y[1] for y in y_pred_unscaled]
  WD_cos_pred_unscaled = [y[2] for y in y_pred_unscaled]
  WD_pred = [np.degrees(math.atan2(sincos[0], sincos[1])) for sincos in zip(WD_sin_pred_unscaled, WD_cos_pred_unscaled)]
  for i in range(len(WD_pred)): 
    if WD_pred[i]<=0: WD_pred[i] += 360

  WD_sin_test_unscaled = [y[1] for y in y_test_unscaled]
  WD_cos_test_unscaled = [y[2] for y in y_test_unscaled]
  WD_test = [np.degrees(math.atan2(sincos[0], sincos[1])) for sincos in zip(WD_sin_test_unscaled, WD_cos_test_unscaled)]
  for i in range(len(WD_test)): 
    if WD_test[i]<=0: WD_test[i] += 360

  WD_base = [y[1] for y in y_base_unscaled]

  return WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base

In [ ]:
def predictions_1p5_h(df_train, df_test, base, title="1.5 hours prediction"):
  scaler = MinMaxScaler()
  scaler_base = MinMaxScaler()

  X_train, y_train = get_data(df_train, scaler) 
  X_test, y_test = get_data(df_test, scaler)
  y_base = get_baseline(base, scaler_base)

  # hiperparameters: results of optimization
  model = MLPRegressor(
          hidden_layer_sizes=(81),   
          activation='relu',
          solver='adam',
          random_state=42,
          max_iter=100,
          batch_size=48)

  print("\n================= " + title + " ========================")

  print("Model training...")
  model.fit(X_train, y_train)

  print("Model testing...")
  WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base = make_prediction(model, scaler, X_test, y_test, scaler_base, y_base)
  print("Done ! --------------------------------\n")

  return WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base


In [ ]:
def predictions_3_h(df_train, df_test, base, title="3 hours prediction"):
  scaler = MinMaxScaler()
  scaler_base = MinMaxScaler()

  X_train, y_train = get_data(df_train, scaler) 
  X_test, y_test = get_data(df_test, scaler)
  y_base = get_baseline(base, scaler_base)

  # model with hiperparameters: result of selection
  model = MLPRegressor(
          hidden_layer_sizes=(81, 8),   
          activation='relu',
          solver='adam',
          random_state=42,
          max_iter=100,
          batch_size=48)

  print("\n================= " + title + " ========================")

  print("Model training...")
  model.fit(X_train, y_train)

  print("Model testing...")
  WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base = make_prediction(model, scaler, X_test, y_test, scaler_base, y_base)
  print("Done ! --------------------------------\n")

  return WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base

In [ ]:
def predictions_6_h(df_train, df_test, base, title="6 hours prediction"):
  scaler = MinMaxScaler()
  scaler_base = MinMaxScaler()

  X_train, y_train = get_data(df_train, scaler) 
  X_test, y_test = get_data(df_test, scaler)
  y_base = get_baseline(base, scaler_base)

  # model with hiperparameters: result of selection
  model = models.Sequential()
  model.add(layers.Dense(255, activation='relu', input_shape=(225*3,)))
  model.add(layers.Dense(24, activation='relu'))
  model.add(layers.Dense(3))
  model.compile(optimizer='Adam', loss='mean_absolute_error', metrics='mean_absolute_error')

  print("\n================= " + title + " ========================")
  
  print("Model training...")
  model.fit(X_train, y_train, 
            epochs=100,
            batch_size=48*90,
            validation_data=(X_test, y_test),
            verbose=0)

  print("Model testing...")
  WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base = make_prediction(model, scaler, X_test, y_test, scaler_base, y_base)
  print("Done ! --------------------------------\n")

  return WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base

In [ ]:
def predictions_12_h(df_train, df_test, base, title="12 hours prediction"):
  scaler = MinMaxScaler()
  scaler_base = MinMaxScaler()

  X_train, y_train = get_data(df_train, scaler) 
  X_test, y_test = get_data(df_test, scaler)
  y_base = get_baseline(base, scaler_base)

  # model with hiperparameters: result of selection

  size = int(math.sqrt(X_train.shape[1]/3)) # 3 data for each node of square matrix

  X_train = X_train.reshape(X_train.shape[0], size, size, 3)
  X_test = X_test.reshape(X_test.shape[0], size, size, 3)
  
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(size, size, 3)))
  model.add(layers.Flatten())
  model.add(layers.Dense(126, activation='relu', kernel_initializer='he_uniform'))
  model.add(layers.Dense(3))

  model.compile(optimizer='Adam', loss='mean_absolute_error', metrics='mean_absolute_error')

  print("\n================= " + title + " ========================")
  
  print("Model training...")
  model.fit(X_train, y_train, 
            epochs=60,
            batch_size=48*60,
            validation_data=(X_test, y_test),
            verbose=0)

  print("Model testing...")
  WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base = make_prediction(model, scaler, X_test, y_test, scaler_base, y_base)
  print("Done ! --------------------------------\n")

  return WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base

# Functions for results displaying

In [ ]:
from IPython.display import display
from ipywidgets import GridspecLayout, Output, Tab, Dropdown, FloatSlider, Button, HTML, Layout

from datetime import datetime, timedelta
from matplotlib.animation import FuncAnimation
from matplotlib.animation import writers
from matplotlib import rc
rc('animation', html='jshtml')

In [ ]:
def show_results(WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base):

  print("\r----------------- TEST DATA -----------------------------------------------", end="")
  print(f"\nWIND SPEED         (m/s) - Min: {round(min(WS_test_unscaled),4)}, Max: {round(max(WS_test_unscaled),4)}, Mean: {round(sum(WS_test_unscaled)/len(WS_test_unscaled),4)}")
  print(f"WIND DIRECTION     (deg) - Min: {round(min(WD_test),4)}, Max: {round(max(WD_test),4)}, Mean: {round(sum(WD_test)/len(WD_test),4)}")

  print("\n----------------- UNSCALED WIND SPEED METRICS -----------------------------")
  print(f"MAE OF PREDICTION  (m/s) : {round(mean_absolute_error(WS_test_unscaled, WS_pred_unscaled),4)}")
  print(f"MAE OF BASELINE    (m/s) : {round(mean_absolute_error(WS_test_unscaled, WS_base_unscaled),4)}")
  print(f"MAPE OF PREDICTION (%)   : {round(mean_absolute_percentage_error(WS_test_unscaled, WS_pred_unscaled),4)}")
  print(f"MAPE OF BASELINE   (%)   : {round(mean_absolute_percentage_error(WS_test_unscaled, WS_base_unscaled),4)}")

  print("\n----------------- UNSCALED WIND DIRECTION METRICS --------------------------")
  print(f"MAE OF PREDICTION  (deg) : {round(MAE_deg(WD_test, WD_pred),4)}")
  print(f"MAE OF BASELINE    (deg) : {round(MAE_deg(WD_test, WD_base),4)}")
  print(f"MAPE OF PREDICTION (%)   : {round(MAPE_deg(WD_test, WD_pred),4)}")
  print(f"MAPE OF BASELINE   (%)   : {round(MAPE_deg(WD_test, WD_base),4)}")

In [ ]:
def show_winds(length_list_test, angles_list_test, length_list_pred, angles_list_pred, interval=20, title="Winds comparison"):

  circle_ray = max(max(length_list_test), max(length_list_pred))+0.5
  circle_angles = [np.radians(angle_deg) for angle_deg in range(360)]
  x_circle_list = [circle_ray * math.sin(angle) for angle in circle_angles]
  y_circle_list = [circle_ray * math.cos(angle) for angle in circle_angles]
  circle_color = 'blue'
  circle_width = 2

  fig_range=(-circle_ray-1.5, circle_ray+1.5)

  arrow_width_t = 2
  arrow_color_t = 'red'
  arrow_alpha_t = 1
  arrowhead_par_t = {'width': arrow_width_t,
                    'headwidth': arrow_width_t + 4,
                    'headlength': 10,
                     'color': arrow_color_t,
                     'alpha': arrow_alpha_t}
  
  arrow_width_p = 10
  arrow_color_p = 'green'
  arrow_alpha_p = 0.4
  arrowhead_par_p = {'width': arrow_width_p,
                    'headwidth': arrow_width_p + 4,
                    'headlength': 10,
                     'color': arrow_color_p,
                     'alpha': arrow_alpha_p}

  fig, ax = plt.subplots(figsize=(6, 6))
  plt.axis('off')
  ax.set_title(title)

  ax.set_xlim(fig_range)
  ax.set_ylim(fig_range)

  ax.scatter(x_circle_list, y_circle_list, s=circle_width, color=circle_color)

  ax.plot([0,0], [circle_ray, circle_ray+0.2], color=circle_color)
  ax.text(-0.2, circle_ray+0.4, 'N', color=circle_color, fontsize=16)

  ax.plot([0,0], [-circle_ray, -circle_ray-0.2], color=circle_color)
  ax.text(-0.2, -circle_ray-0.8, 'S', color=circle_color, fontsize=16)

  ax.plot([-circle_ray, -circle_ray-0.2], [0,0], color=circle_color)
  ax.text(-circle_ray-0.8, -0.2, 'W',color=circle_color, fontsize=16)

  ax.plot([circle_ray, circle_ray+0.2], [0,0], color=circle_color)
  ax.text(circle_ray+0.3, -0.2, 'E', color=circle_color, fontsize=16)

  arrow_p, = ax.plot([0,0], [0,0], lw=arrow_width_p, color=arrow_color_p, alpha=arrow_alpha_p)
  arrow_p.set_label("predicted data")
  arrowhead_p = ax.annotate('', xytext=(0,0), xy=(0,0), arrowprops=arrowhead_par_p)

  arrow_t, = ax.plot([0,0], [0,0], lw=arrow_width_t, color=arrow_color_t, alpha=arrow_alpha_t)
  arrow_t.set_label("test data")
  arrowhead_t = ax.annotate('', xytext=(0,0), xy=(0,0), arrowprops=arrowhead_par_t)
  
  plt.legend()

  def plot_winds(i):
  
    x_p = length_list_pred[i] * math.sin(np.radians(angles_list_pred[i]))
    y_p = length_list_pred[i] * math.cos(np.radians(angles_list_pred[i]))
    x_head_p = (length_list_pred[i] + 0.3) * math.sin(np.radians(angles_list_pred[i]))
    y_head_p = (length_list_pred[i] + 0.3) * math.cos(np.radians(angles_list_pred[i]))

    arrow_p.set_data([0, x_p], [0, y_p])

    arrowhead_p.set_position((x_p, y_p))
    arrowhead_p.xy = (x_head_p, y_head_p)

    x_t = length_list_test[i] * math.sin(np.radians(angles_list_test[i]))
    y_t = length_list_test[i] * math.cos(np.radians(angles_list_test[i]))
    x_head_t = (length_list_test[i] + 0.3) * math.sin(np.radians(angles_list_test[i]))
    y_head_t = (length_list_test[i] + 0.3) * math.cos(np.radians(angles_list_test[i]))

    arrow_t.set_data([0, x_t], [0, y_t])

    arrowhead_t.set_position((x_t, y_t))
    arrowhead_t.xy = (x_head_t, y_head_t)

    return arrow_t, arrowhead_t, arrow_p, arrowhead_p

  # frames = min(len(length_list_test), len(length_list_pred))
  frames = 7*48  # only 1 week
  wind_animation = FuncAnimation(fig, plot_winds, frames=frames, interval=interval, blit=True)
  plt.close()

  return wind_animation

In [ ]:
def plot_wind_speed(year, WS_pred, WS_test, WS_base, pred_steps):

  start_hour = datetime.strptime(str(year)+"-01-01 0:00", '%Y-%m-%d %H:%M')+timedelta(minutes=pred_steps*30)
  datetimes = pd.date_range(start_hour.strftime('%Y-%m-%d %H:%M'), str(year)+"-12-31 23:30", freq="30min")

  # only 1 week
  plt.figure(figsize=(18, 5))
  plt.plot(datetimes[:7*48], WS_pred[:7*48], color='green', linewidth=1)
  plt.plot(datetimes[:7*48], WS_test[:7*48], color='red', linewidth=1)
  plt.plot(datetimes[:7*48], WS_base[:7*48], color='black', linewidth=1)
  plt.xlim(datetimes[:7*48].min(), datetimes[:7*48].max())
  plt.title('WIND SPEED')
  plt.ylabel('[m/s]')
  plt.xlabel('time')
  plt.legend(['prediction', 'test data', 'baseline'], loc='upper left')
  plt.show()

In [ ]:
def plot_wind_direction(year, WD_pred, WD_test, WD_base, pred_steps):

  start_hour = datetime.strptime(str(year)+"-01-01 0:00", '%Y-%m-%d %H:%M')+timedelta(minutes=pred_steps*30)
  datetimes = pd.date_range(start_hour.strftime('%Y-%m-%d %H:%M'), str(year)+"-12-31 23:30", freq="30min")

  # only 1 week
  plt.figure(figsize=(18, 5))
  plt.plot(datetimes[:7*48], WD_pred[:7*48], color='green', linewidth=1)
  plt.plot(datetimes[:7*48], WD_test[:7*48], color='red', linewidth=1)
  plt.plot(datetimes[:7*48], WD_base[:7*48], color='black', linewidth=1)
  plt.xlim(datetimes[:7*48].min(), datetimes[:7*48].max())
  plt.title('WIND DIRECTION')
  plt.ylabel('[deg]')
  plt.xlabel('time')
  plt.legend(['prediction', 'test data', 'baseline'], loc='upper left')
  plt.show()

In [ ]:
def speed_test_dashboard(WS_test, style):
  ws_min = str(round(min(WS_test),4))
  ws_max = str(round(max(WS_test),4))
  ws_avg = str(round(sum(WS_test)/len(WS_test),4))

  html = style
  html += '<table>'
  html += '  <tr>'
  html += '    <td>'
  html += '      <p class="tit-gen"> SPEED data </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Min </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Max </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Average </p>'
  html += '    </td>'
  html += '  </tr>'
  html += '  <tr>'
  html += '    <td>'
  html += '      <p class="tit-left"> Test dataset: </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+ws_min+'</p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+ws_max+'</p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+ws_avg+'</p>'
  html += '    </td>'
  html += '  </tr>'
  html += '</table>'
  
  return html

In [ ]:
def direction_test_dashboard(WD_test, style):
  wd_min = str(round(min(WD_test),4))
  wd_max = str(round(max(WD_test),4))
  wd_avg = str(round(sum(WD_test)/len(WD_test),4))

  html = style
  html += '<table>'
  html += '  <tr>'
  html += '    <td>'
  html += '      <p class="tit-gen"> DIRECTION data </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Min </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Max </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Average </p>'
  html += '    </td>'
  html += '  </tr>'
  html += '  <tr>'
  html += '    <td>'
  html += '      <p class="tit-left"> Test dataset: </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+wd_min+'</p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+wd_max+'</p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+wd_avg+'</p>'
  html += '    </td>'
  html += '  </tr>'
  html += '</table>'
  
  return html

In [ ]:
def speed_prediction_dashboard(WS_test, WS_pred, WS_base, style):

  mae_pred = round(mean_absolute_error(WS_test, WS_pred),4)
  mae_base = round(mean_absolute_error(WS_test, WS_base),4)
  mape_pred = round(mean_absolute_percentage_error(WS_test, WS_pred),4)
  mape_base = round(mean_absolute_percentage_error(WS_test, WS_base),4)
  
  mae_pred_str = str(mae_pred)
  mae_base_str = str(mae_base)
  mape_pred_str = str(mape_pred)
  mape_base_str = str(mape_base)

  mae_dif_str = str(round(mae_pred-mae_base,4))
  mape_dif_str = str(round(mape_pred-mape_base,4))

  html = style
  html += '<table>'
  html += '  <tr>'
  html += '    <td>'
  html += '      <p class="tit-gen"> SPEED Prediction </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Model </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Baseline </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Difference </p>'
  html += '    </td>'
  html += '  </tr>'
  html += '  <tr>'
  html += '    <td>'
  html += '      <p class="tit-left"> MAE: </p>'
  html += '      <p class="tit-left"> MAPE: </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+mae_pred_str+'</p>'
  html += '      <p class="var">'+mape_pred_str+'</p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+mae_base_str+'</p>'
  html += '      <p class="var">'+mape_base_str+'</p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+mae_dif_str+'</p>'
  html += '      <p class="var">'+mape_dif_str+'</p>'
  html += '    </td>'
  html += '  </tr>'
  html += '</table>'

  return html

In [ ]:
def direction_prediction_dashboard(WD_test, WD_pred, WD_base, style):

  mae_pred = round(MAE_deg(WD_test, WD_pred),4)
  mae_base = round(MAE_deg(WD_test, WD_base),4)
  mape_pred = round(MAPE_deg(WD_test, WD_pred),4)
  mape_base = round(MAPE_deg(WD_test, WD_base),4)

  mae_pred_str = str(mae_pred)
  mae_base_str = str(mae_base)
  mape_pred_str = str(mape_pred)
  mape_base_str = str(mape_base)

  mae_dif_str = str(round(mae_pred-mae_base,4))
  mape_dif_str = str(round(mape_pred-mape_base,4))

  html = style
  html += '<table>'
  html += '  <tr>'
  html += '    <td>'
  html += '      <p class="tit-gen"> DIREC. Prediction </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Model </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Baseline </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="tit-up"> Difference </p>'
  html += '    </td>'
  html += '  </tr>'
  html += '  <tr>'
  html += '    <td>'
  html += '      <p class="tit-left"> MAE: </p>'
  html += '      <p class="tit-left"> MAPE: </p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+mae_pred_str+'</p>'
  html += '      <p class="var">'+mape_pred_str+'</p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+mae_base_str+'</p>'
  html += '      <p class="var">'+mape_base_str+'</p>'
  html += '    </td>'
  html += '    <td>'
  html += '      <p class="var">'+mae_dif_str+'</p>'
  html += '      <p class="var">'+mape_dif_str+'</p>'
  html += '    </td>'
  html += '  </tr>'
  html += '</table>'

  return html

In [ ]:
def show_dashboard(WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base, test_year, pred_steps, title):
  grid = GridspecLayout(30, 41)
  
  html = '<div style="font-size: 30px; font-weight: bold; color: black;"><center>'+"WIND PREDICTION RESULTS: "+title+'</center></div>'
  header = HTML(html)

  html =  '<style>'
  html += '  table {border-spacing: 10px; background-color: Silver; width: 100%; margin-bottom: 10px; border-left: 5px solid Gainsboro; border-top: 5px solid Gainsboro; border-right: 5px solid gray; border-bottom: 5px solid gray;}'
  html += '  td {vertical-align: middle; background-color: Silver; width: 25%; text-align: center;}'
  html += '  .tit-up {font-weight: bold; font-size: 15px; color: black; text-align: center;}'
  html += '  .tit-left {font-weight: bold; font-size: 15px; color: black; text-align: left;}'
  html += '  .tit-gen {font-weight: bold; font-size: 15px; color: blue; text-align: left;}'
  html += '  .var {font-weight: bold; font-size: 15px; color: red; border-left: 1px solid gray; border-top: 1px solid gray; border-right: 1px solid Gainsboro; border-bottom: 1px solid Gainsboro}'
  html += '</style>'
  
  style = html

  html = speed_test_dashboard(WS_test_unscaled, style)
  speed_test = HTML(html)

  html = speed_prediction_dashboard(WS_test_unscaled, WS_pred_unscaled, WS_base_unscaled, style)
  speed_pred = HTML(html)

  html = direction_test_dashboard(WD_test, style)
  direc_test = HTML(html)

  html = direction_prediction_dashboard(WD_test, WD_pred, WD_base, style)
  direc_pred = HTML(html)

  speed_plot = Output(layout=Layout(display="flex", justify_content="center"))
  with speed_plot:
    plot_wind_speed(test_year, WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, pred_steps)

  direc_plot = Output(layout=Layout(display="flex", justify_content="center"))
  with direc_plot:
    plot_wind_direction(test_year, WD_pred, WD_test, WD_base, pred_steps)

  
  wind = show_winds(WS_test_scaled, WD_test, WS_pred_scaled, WD_pred, title=title)
    
  grid[1,:] = header
  grid[2:5,1:20] = speed_test
  grid[6:10,1:20] = speed_pred

  grid[2:5,21:40] = direc_test
  grid[6:10,21:40] = direc_pred

  grid[11:18, 1:40] = speed_plot
  grid[19:30, 1:40] = direc_plot

  display(grid)

  return wind

# Functions for parameters input and program launching

In [ ]:
def input_panel():

  html = '<div style="font-size: 30px; font-weight: bold; color: black;"><center> WIND PREDICTION PARAMETERS </center></div>'
  header = HTML(html)

  html = '<div style="font-size: 15px; color: blue; font-weight: bold;"> Select location for wind prediction </div>'
  title_locations = HTML(html)

  locations = {}
  locations[0] = ("Not selected", 0, 0)
  locations[1] = ("Łódź N-51.7666 E-19.4604", 51.7666, 19.4604)
  locations[2] = ("Katowice N-50.2602 E-19.0321", 50.2602, 19.0321)
  locations[3] = ("Gdynia N-54.5223 E-18.5486", 54.5223, 18.5486)
  locations[4] = ("Bałtyk N-55.3326 E-18.0680", 55.3326, 18.0680)
  locations[5] = ("Śniardwy N-53.7549 E-21.7179", 53.7549, 21.7179)
  locations[6] = ("Puszcza Białowieska N-52.8026 E-23.8395", 52.8026, 23.8395)
  locations[7] = ("Łysa Góra N-50.5137 E-21.0249", 50.5137, 21.0249)
  locations[8] = ("Rysy N-49.1077 E-20.0528", 49.1077, 20.0528)
  locations[9] = ("Orle Gniazda N-50.5637 E-19.5126", 50.5637, 19.5126)
  locations[10] = ("Żuławy Wiślane N-54.1000 E-19.0000", 54.1000, 19.0000)

  options = []
  for i in range(len(locations)):
    options.append((locations[i][0], i))

  list_locations = Dropdown(
      options=options,
      value=0,
  )

  html = '<div style="font-size: 15px; color: blue; font-weight: bold;"> Or input coordinates </div>'
  title_coordinates = HTML(html)

  input_lat = FloatSlider(
    value=54,
    min=35,
    max=71,
    step=0.0001,
    description='Latitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.4f',
    layout=Layout(width="100%")
  )

  input_lon = FloatSlider(
    value=3,
    min=0,
    max=24,
    step=0.0001,
    description='Longitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.4f',
    layout=Layout(width="100%")
  )

  html = '<div style="font-size: 15px; color: blue; font-weight: bold;"> Select years for prediction </div>'
  title_years = HTML(html)

  input_train = Dropdown(
    options=[y for y in range(1989, 2019)],
    value=2017,
    description='Training:'
  )

  input_test = Dropdown(
    options=[y for y in range(1989, 2019)],
    value=2018,
    description='Testing:'
  )

  enter = Button(
    description='Get data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Get data'
)

  grid = GridspecLayout(10, 10)

  grid[1,:]   = header

  grid[3,1:3] = title_locations
  grid[3,4:6] = list_locations

  grid[5,1:3] = title_coordinates
  grid[5,4:6] = input_lat
  grid[5,7:9] = input_lon

  grid[7,1:3] = title_years
  grid[7,4:6] = input_train
  grid[7,7:9] = input_test

  display(grid)

  return locations, list_locations, input_lat, input_lon, input_train, input_test

In [ ]:
def get_input_params(locations, list_locations, input_lat, input_lon, input_train, input_test):
  listloc = list_locations.value
  if listloc==0:
    lat = input_lat.value
    lon = input_lon.value
  else:
    lat = locations[listloc][1]
    lon = locations[listloc][2]
  train_yr = input_train.value
  test_yr = input_test.value

  params={}
  params['LAT']=lat
  params['LON']=lon
  params['TRAIN-YEAR']=train_yr
  params['TEST-YEAR']=test_yr
  
  return params

In [ ]:
def run_program(locations, list_locations, input_lat, input_lon, input_train, input_test):

  params = get_input_params(locations, list_locations, input_lat, input_lon, input_train, input_test)

  radius_km = 23  # around location of prediction

  dataset_train = get_wind_data(params['LAT'], params['LON'], radius_km, params['TRAIN-YEAR'], "LOADING TRAINING DATA")
  dataset_test = get_wind_data(params['LAT'], params['LON'], radius_km, params['TEST-YEAR'], "LOADING TESTING DATA")

  df_train_prep = prepare_dataframe(dataset_train, "PREPROCESSING TRAINING DATA")
  df_test_prep = prepare_dataframe(dataset_test, "PREPROCESSING TESTING DATA")

  y_node = (7, 7) # center of nodes matrix, selected location

  df_train_1, df_train_3, df_train_6, df_train_12 = create_prediction_datasets(df_train_prep, y_node, "PREPARING TRAINING DATASETS")
  df_test_1, df_test_3, df_test_6, df_test_12, base_1, base_3, base_6, base_12 = create_prediction_datasets(df_test_prep, y_node, "PREPARING TESTING DATASETS", baseline=True)

  anim_list=[]

  # making and showing predictions for 1.5 hour ahead (3 steps)
  WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base = predictions_1p5_h(df_train_1, df_test_1, base_1, title="1.5 hours prediction")
  anim = show_dashboard(WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base, params['TEST-YEAR'], 3, title="1.5 hours ahead")
  anim_list.append(anim)

  # making and showing predictions for 3 hours ahead (6 steps)
  WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base = predictions_3_h(df_train_3, df_test_3, base_3, title="3 hours prediction")
  anim = show_dashboard(WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base, params['TEST-YEAR'], 6, title="3 hours ahead")
  anim_list.append(anim)

  # making and showing predictions for 6 hours ahead (12 steps)
  WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base = predictions_6_h(df_train_6, df_test_6, base_6, title="6 hours prediction")
  anim = show_dashboard(WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base, params['TEST-YEAR'], 12, title="6 hours ahead")
  anim_list.append(anim)

  # making and showing predictions for 12 hours ahead (24 steps)
  WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base = predictions_12_h(df_train_12, df_test_12, base_12, title="12 hours prediction")
  anim = show_dashboard(WS_pred_unscaled, WS_test_unscaled, WS_base_unscaled, WS_pred_scaled, WS_test_scaled, WS_base_scaled, WD_pred, WD_test, WD_base, params['TEST-YEAR'], 24, title="12 hours ahead")
  anim_list.append(anim)
  
  return anim_list


# Program launching

In [ ]:
locations, list_locations, input_lat, input_lon, input_train, input_test = input_panel()

In [ ]:
anim_list = run_program(locations, list_locations, input_lat, input_lon, input_train, input_test)

In [ ]:
anim_list[0]

In [ ]:
anim_list[1]

In [ ]:
anim_list[2]

In [ ]:
anim_list[3]